
# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
#update automl
!pip install --upgrade azureml-sdk[automl]

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

import pandas as pd
from train import clean_data
from azureml.train.automl import AutoMLConfig

from azureml.core import Workspace, Dataset

from azureml.core.model import Model

from azureml.core.resource_configuration import ResourceConfiguration

import pandas as pd
from train import clean_data
from azureml.train.automl import AutoMLConfig

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
import azureml.train.automl
import azureml.automl

import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is extracted from an Udacity's competition called 'Predicting customer satisfaction' It consists of customers data from Santander bank, containing all their relevant features (like the number of products, their name, interest rate of loans, etc). 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
cluster_name = 'project-cluster'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
#import dataset
Data_URL= "https://github.com/jfcarmonag/Capstone_AZMLND/raw/master/starter_file/BankChurners.csv"
ds=TabularDatasetFactory.from_delimited_files(path=Data_URL)
# choose a name for experiment
experiment_name = 'exp-capstone'
experiment=Experiment(ws, experiment_name)

In [ ]:
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
complete_data = pd.concat([x, y], axis=1)

complete_data

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.



In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

local_path = 'data/prepared.csv'
complete_data.to_csv(local_path, index=False)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/prepared.csv'))])

In [ ]:


# automl settings
automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'norm_macro_recall',
       "enable_early_stopping": True,
       "experiment_timeout_hours": 1.0,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       #"verbosity": logging.INFO,
   }

# automl configuration
automl_config = AutoMLConfig(task = 'classification',
                               compute_target = compute_target,
                               training_data = dataset,
                               label_column_name = "Attrition_Flag",
                               **automl_settings
                               )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output="True")

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = remote_run.get_output()

In [ ]:
import joblib
joblib.dump(value=best_model,filename='model.pkl')

In [ ]:
#TODO: Save the best model
os.makedirs('outputs',exist_ok=True)
model = best_run.register_model(model_name = "model", model_path = './outputs/model.pkl', 
                                model_framework= Model.Framework.SCIKITLEARN, model_framework_version='0.19.1')
                                #resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

In [ ]:
best_run.get_file_names()

In [ ]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')

In [ ]:
#from azureml.automl.runtime.onnx_convert import OnnxConverter
#best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)
#onnx_fl_path = "./best_model.onnx"
#OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:


env_name="environment1"
#Use curated environment AzureML-Minimal
env = Environment.get(ws, "AzureML-AutoML")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',environment=env)

depl_config = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=2, enable_app_insights=True, auth_enabled= True)

service = Model.deploy(ws, name="myservice1", models=[model], inference_config=inference_config, deployment_config=depl_config)
#service = Model.deploy(ws, "myservice1", [model], deployment_config=depl_config)
#service = Model.deploy(ws, "myservice1", [model])

service.wait_for_deployment(show_output = True)
#enable logging
service.update(enable_app_insights=True)
print(service.get_logs())
print(service.state)



TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
primary, secondary = service.get_keys()
print(primary)
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
          #"Column1": 0,
          "Customer_Age": 45,
          "Gender_M":1,
          "Gender_F":0,
          "Dependent_count":3,
          "Education_Level":2,
          "Married":0,
          "Single":1,
          "Divorced":0,
          "Unknown":0,
          "Income_Category":2,
          "Card_Category":2,
          "Months_on_book":39,
          "Total_Relationship_Count":5,
          "Months_Inactive_12_mon":1,
          "Contacts_Count_12_mon":3,
          "Credit_Limit":12691,
          "Total_Revolving_Bal":777,
          "Avg_Open_To_Buy":11914,
          "Total_Amt_Chng_Q4_Q1":1.335,
          "Total_Trans_Amt":1144,
          "Total_Trans_Ct":42,
          "Total_Ct_Chng_Q4_Q1":1.625,
          "Avg_Utilization_Ratio":0.061,
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
print(logs)
service.delete()